<a href="https://colab.research.google.com/github/jesseryantlg/2015/blob/master/DS484_PE10_JesseRyan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!pip install pyspark
!pip install findspark

#importing the necessary modules
import os
import sys
import pyspark
import random
import pandas as pd
import findspark
findspark.init()

from pyspark.ml.regression import LinearRegression
from pyspark.ml.stat import Correlation
import pyspark.sql.functions as F

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"

In [5]:
#Import SparkSession module

from pyspark.sql import SparkSession

#Creating sparksession object
spark = SparkSession.builder.master("local[*]").appName("Washington State housing price analysis").getOrCreate()

#Call the spark session variable object

spark

In [9]:
#importing data
house_data = spark.read.csv("WA_house_data.csv", inferSchema = True, header = True)

In [10]:
house_data.show(truncate=True)

+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+------------+-------+--------+-------------+----------+
|    price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|        city|    lat|    long|sqft_living15|sqft_lot15|
+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+------------+-------+--------+-------------+----------+
| 221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|     Seattle|47.5112|-122.257|         1340|      5650|
| 538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|        1991|  98125|     Seattle| 47.721|-122.319|         1690|      7

In [11]:
#Check the entire dataset
house_data.head(5)

[Row(price=221900.0, bedrooms=3, bathrooms=1.0, sqft_living=1180, sqft_lot=5650, floors=1.0, waterfront=0, view=0, condition=3, grade=7, sqft_above=1180, sqft_basement=0, yr_built=1955, yr_renovated=0, zipcode=98178, city='Seattle', lat=47.5112, long=-122.257, sqft_living15=1340, sqft_lot15=5650),
 Row(price=538000.0, bedrooms=3, bathrooms=2.25, sqft_living=2570, sqft_lot=7242, floors=2.0, waterfront=0, view=0, condition=3, grade=7, sqft_above=2170, sqft_basement=400, yr_built=1951, yr_renovated=1991, zipcode=98125, city='Seattle', lat=47.721, long=-122.319, sqft_living15=1690, sqft_lot15=7639),
 Row(price=180000.0, bedrooms=2, bathrooms=1.0, sqft_living=770, sqft_lot=10000, floors=1.0, waterfront=0, view=0, condition=3, grade=6, sqft_above=770, sqft_basement=0, yr_built=1933, yr_renovated=0, zipcode=98028, city='Kenmore', lat=47.7379, long=-122.233, sqft_living15=2720, sqft_lot15=8062),
 Row(price=604000.0, bedrooms=4, bathrooms=3.0, sqft_living=1960, sqft_lot=5000, floors=1.0, waterf

In [12]:
#Check the schema
house_data.printSchema()

root
 |-- price: double (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



In [13]:
#Import module
from pyspark.ml.feature import StringIndexer

#Convert string to integer
house = StringIndexer(inputCol = 'city', outputCol = 'city_int')
house_indexed = house.fit(house_data).transform(house_data)

In [14]:
#Check the entire dataset
house_indexed.show()

+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+------------+-------+--------+-------------+----------+--------+
|    price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|        city|    lat|    long|sqft_living15|sqft_lot15|city_int|
+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+------------+-------+--------+-------------+----------+--------+
| 221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|     Seattle|47.5112|-122.257|         1340|      5650|     0.0|
| 538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|        1991|  98125|     Seattle| 4

In [16]:
#Converting columns into independent features (variable)
#exclude string data type
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols=["bedrooms", "bathrooms", "sqft_lot", "floors", "waterfront", "view", "condition", "grade", "sqft_above", "sqft_basement", "yr_built", "yr_renovated", "zipcode", "city_int", "sqft_living15", "sqft_lot15"], outputCol = "Independent Features")

In [17]:
output = featureassembler.transform(house_indexed)

In [18]:
output.show()

+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+------------+-------+--------+-------------+----------+--------+--------------------+
|    price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|        city|    lat|    long|sqft_living15|sqft_lot15|city_int|Independent Features|
+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+------------+-------+--------+-------------+----------+--------+--------------------+
| 221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|     Seattle|47.5112|-122.257|         1340|      5650|     0.0|[3.0,1.0,5650.0,1...|
| 538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|  

In [19]:
output.select("Independent Features").show(truncate=False)

+--------------------------------------------------------------------------------------------+
|Independent Features                                                                        |
+--------------------------------------------------------------------------------------------+
|[3.0,1.0,5650.0,1.0,0.0,0.0,3.0,7.0,1180.0,0.0,1955.0,0.0,98178.0,0.0,1340.0,5650.0]        |
|[3.0,2.25,7242.0,2.0,0.0,0.0,3.0,7.0,2170.0,400.0,1951.0,1991.0,98125.0,0.0,1690.0,7639.0]  |
|[2.0,1.0,10000.0,1.0,0.0,0.0,3.0,6.0,770.0,0.0,1933.0,0.0,98028.0,13.0,2720.0,8062.0]       |
|[4.0,3.0,5000.0,1.0,0.0,0.0,5.0,7.0,1050.0,910.0,1965.0,0.0,98136.0,0.0,1360.0,5000.0]      |
|[3.0,2.0,8080.0,1.0,0.0,0.0,3.0,8.0,1680.0,0.0,1987.0,0.0,98074.0,7.0,1800.0,7503.0]        |
|[4.0,4.5,101930.0,1.0,0.0,0.0,3.0,11.0,3890.0,1530.0,2001.0,0.0,98053.0,4.0,4760.0,101930.0]|
|[3.0,2.25,6819.0,2.0,0.0,0.0,3.0,7.0,1715.0,0.0,1995.0,0.0,98003.0,8.0,2238.0,6819.0]       |
|[3.0,1.5,9711.0,1.0,0.0,0.0,3.0,7.0,1060.0,0.0,19

In [20]:
output.columns

['price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'city',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'city_int',
 'Independent Features']

In [21]:
#Selecting independent (feature) and dependent variable (price)
finalized_data = output.select("Independent Features", "price")

In [22]:
finalized_data.show()

+--------------------+---------+
|Independent Features|    price|
+--------------------+---------+
|[3.0,1.0,5650.0,1...| 221900.0|
|[3.0,2.25,7242.0,...| 538000.0|
|[2.0,1.0,10000.0,...| 180000.0|
|[4.0,3.0,5000.0,1...| 604000.0|
|[3.0,2.0,8080.0,1...| 510000.0|
|[4.0,4.5,101930.0...|1230000.0|
|[3.0,2.25,6819.0,...| 257500.0|
|[3.0,1.5,9711.0,1...| 291850.0|
|[3.0,1.0,7470.0,1...| 229500.0|
|[3.0,2.5,6560.0,2...| 323000.0|
|[3.0,2.5,9796.0,1...| 662500.0|
|[2.0,1.0,6000.0,1...| 468000.0|
|[3.0,1.0,19901.0,...| 310000.0|
|[3.0,1.75,9680.0,...| 400000.0|
|[5.0,2.0,4850.0,1...| 530000.0|
|[4.0,3.0,5000.0,2...| 650000.0|
|[3.0,2.0,14040.0,...| 395000.0|
|[4.0,1.0,4300.0,1...| 485000.0|
|[2.0,1.0,9850.0,1...| 189000.0|
|[3.0,1.0,9774.0,1...| 230000.0|
+--------------------+---------+
only showing top 20 rows



In [23]:
#Split the data into train & test
train_data, test_data = finalized_data.randomSplit([0.70, 0.30])

In [24]:
train_data.show()

+--------------------+--------+
|Independent Features|   price|
+--------------------+--------+
|[0.0,0.0,979.0,3....|380000.0|
|[0.0,0.0,4269.0,1...|139950.0|
|[0.0,0.0,4800.0,2...|235000.0|
|[0.0,0.0,20875.0,...|142000.0|
|[0.0,0.75,213444....|265000.0|
|[0.0,1.0,5900.0,1...|228000.0|
|[0.0,2.5,5669.0,2...|240000.0|
|[1.0,0.0,23244.0,...|484000.0|
|[1.0,0.0,43377.0,...| 75000.0|
|[1.0,0.75,1801.0,...|276000.0|
|[1.0,0.75,2885.0,...|310000.0|
|[1.0,0.75,5050.0,...| 80000.0|
|[1.0,0.75,5600.0,...|170000.0|
|[1.0,0.75,5650.0,...|202000.0|
|[1.0,0.75,6034.0,...|272000.0|
|[1.0,0.75,8636.0,...|325000.0|
|[1.0,0.75,9750.0,...|145000.0|
|[1.0,0.75,10079.0...|369900.0|
|[1.0,0.75,12120.0...|299000.0|
|[1.0,0.75,12981.0...|262000.0|
+--------------------+--------+
only showing top 20 rows



In [25]:
test_data.show()

+--------------------+---------+
|Independent Features|    price|
+--------------------+---------+
|[0.0,0.0,4764.0,3...|1100000.0|
|[0.0,0.0,8049.0,2...| 355000.0|
|[0.0,0.0,28008.0,...|1300000.0|
|[0.0,1.5,1650.0,3...| 288000.0|
|[0.0,2.5,7111.0,2...| 320000.0|
|[0.0,2.5,8319.0,2...| 339950.0|
|[1.0,0.0,24501.0,...| 280000.0|
|[1.0,0.5,1642.0,1...| 255000.0|
|[1.0,0.75,1284.0,...| 290000.0|
|[1.0,0.75,4600.0,...| 315000.0|
|[1.0,0.75,5746.0,...|  95000.0|
|[1.0,0.75,6600.0,...| 351000.0|
|[1.0,0.75,6720.0,...| 280000.0|
|[1.0,0.75,7203.0,...| 124000.0|
|[1.0,0.75,7203.0,...| 224000.0|
|[1.0,0.75,9600.0,...| 200000.0|
|[1.0,0.75,59677.0...| 527550.0|
|[1.0,0.75,77603.0...| 190000.0|
|[1.0,0.75,87120.0...| 250000.0|
|[1.0,1.0,833.0,1....| 202000.0|
+--------------------+---------+
only showing top 20 rows



In [26]:
#Applying linear regression on selected features and label data
regressor = LinearRegression(featuresCol = "Independent Features", labelCol ="price")

In [27]:
#training our model
trained_model = regressor.fit(train_data)

In [28]:
#Evaluating the trained model with train data
train_results = trained_model.evaluate(train_data)

In [30]:
#Calculating Rsquared and printing value
print("The value of Rsquared is: ", train_results.r2)
print("The model accuracy is {0:.0f}% with train data".format(train_results.r2*100))

The value of Rsquared is:  0.6610190773575204
The model accuracy is 66% with train data


In [31]:
#Evaluate the trained model on test data
test_results = trained_model.evaluate(test_data)

In [33]:
#Calculating R Squared and printing value
print("The value of Rsquared is: ", test_results.r2)
print("The model accuarcy is {0:.0f}% with test data".format(test_results.r2*100))

The value of Rsquared is:  0.639042567859207
The model accuarcy is 64% with test data


In [34]:
#Predicting the price of housing
predictions = trained_model.transform(test_data)

In [35]:
#display the prediction price and current price
predictions.show(truncate=True)

+--------------------+---------+------------------+
|Independent Features|    price|        prediction|
+--------------------+---------+------------------+
|[0.0,0.0,4764.0,3...|1100000.0| 757204.0039721504|
|[0.0,0.0,8049.0,2...| 355000.0| 645823.9608230554|
|[0.0,0.0,28008.0,...|1300000.0|  1540266.39926368|
|[0.0,1.5,1650.0,3...| 288000.0|388131.55755966157|
|[0.0,2.5,7111.0,2...| 320000.0|  383280.501500573|
|[0.0,2.5,8319.0,2...| 339950.0| 754181.8269468434|
|[1.0,0.0,24501.0,...| 280000.0|-287537.5323210694|
|[1.0,0.5,1642.0,1...| 255000.0|347596.67290125415|
|[1.0,0.75,1284.0,...| 290000.0|296395.10819796845|
|[1.0,0.75,4600.0,...| 315000.0|364131.76134861633|
|[1.0,0.75,5746.0,...|  95000.0|212850.10655846447|
|[1.0,0.75,6600.0,...| 351000.0| 326199.3458735794|
|[1.0,0.75,6720.0,...| 280000.0| 134538.8502517417|
|[1.0,0.75,7203.0,...| 124000.0|225832.44556999952|
|[1.0,0.75,7203.0,...| 224000.0|225832.44556999952|
|[1.0,0.75,9600.0,...| 200000.0| 94967.66970655695|
|[1.0,0.75,5